In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import torch
import numpy as np
import pandas as pd
from glob import glob

from drs import util, eval__

In [ ]:
util.set_seed()

In [ ]:
dir_path = '../save_model/model_file.pth'

test_true = np.load(f"{dir_path}/test_true.npy")
test_pred = np.load(f"{dir_path}/test_pred.npy")

major_ddi_path = f"../data/adj_ddi_major.npy"
moder_ddi_path = f"../data/adj_ddi_moder.npy"

adj_ddi_major = np.load(major_ddi_path)
adj_ddi_moder = np.load(moder_ddi_path)

# load local threshold
local_threshold  = np.load(f"{dir_path}/best_thr_each_class.npy")
local_threshold  = torch.tensor(local_threshold).cuda()
global_threshold = np.load(f"{dir_path}/global_best_threshold.npy").item()

In [ ]:
ci = 95
n_bootstrap = 1000
alpha = (100-ci)/ 2
threshold = local_threshold

In [ ]:
df_global = eval__.metrics_with_ci(
                            torch.tensor(test_pred).cuda(), 
                            torch.tensor(test_true).cuda(),
                            major_adj_ddi_matrix=adj_ddi_major,
                            moder_adj_ddi_matrix=adj_ddi_moder,
                            threshold=threshold,
                            average='micro',
                            n_bootstrap=n_bootstrap,
                            format_4dp=True,   # 👈 4 decimals
                            )

In [ ]:
df_topk = eval__.topk_all_ddi(torch.tensor(test_pred).cuda(),
                torch.tensor(test_true).cuda(),
                major_adj_ddi_matrix=adj_ddi_major,
                moder_adj_ddi_matrix=adj_ddi_moder,
                k_values = [1, 3, 5],
                eps = 1e-8)